In [21]:
import numpy as np
import math
from nltk.tokenize import word_tokenize
from collections import defaultdict
from nltk.translate.metrics import alignment_error_rate
from nltk.metrics.scores import (precision, recall)
from operator import itemgetter

In [3]:
#DATA
file = open('./train.en', 'r')
train_en = file.read()
raw_sentences_train_en = train_en.split("\n")

file = open('./train.hi', 'r')
train_hi = file.read()
sentences_train_hi = train_hi.split("\n")

file = open('./test.en', 'r')
test_en = file.read()
raw_sentences_test_en = test_en.split("\n")

file = open('./test.hi', 'r')
test_hi = file.read()
sentences_test_hi = test_hi.split("\n")

file = open('./dev.en.txt', 'r')
dev_en = file.read()
raw_sentences_dev_en = dev_en.split("\n")

file = open('./dev.hi', 'r')
dev_hi = file.read()
sentences_dev_hi = dev_hi.split("\n")

In [4]:
def make_lower_case(data):
    list_ = []
    for sentence in data:
        list_.append(sentence.lower())
    return list_

In [5]:
sentences_dev_en = make_lower_case(raw_sentences_dev_en)
sentences_test_en = make_lower_case(raw_sentences_test_en)
sentences_train_en = make_lower_case(raw_sentences_train_en)

In [6]:
def is_converged(new, old, epoch):
    epsilone = 0.00000001
#     new = list(new.values())
#     old = list(old.values())
    
#     new_val = []
#     for (hin, ddict) in new.items():
#         for (eng, prob) in ddict.items():
#             new_val.append('{0:.8f}'.format(prob))
            
#     old_val = []
#     for (hin, ddict) in old.items():
#         for (eng, prob) in ddict.items():
#             old_val.append('{0:.8f}'.format(prob))        

#     for i in range(len(old_val)):
#         if math.fabs(float(new_val[i]) - float(old_val[i])) > epsilone: 
    if epoch < 15:
        return False
    return True

In [12]:
def perform_EM(en_sentences, hi_sentences):
    
    uni_ini = 0.00001
    
    translation_prob = defaultdict(lambda: defaultdict(lambda: float(uni_ini)))
    translation_prob_prev = defaultdict(lambda: defaultdict(float))
    
    epoch = 0
    
    while True:

        epoch += 1
        print("epoch num:", epoch,"\n")
        count = defaultdict(float)
        total = defaultdict(float)
        for index_sen, hin_sen in enumerate(hi_sentences):
            #compute normalization
            hin_sen_words = hin_sen.split(" ")
            s_total = defaultdict(float)
            for hin_word in hin_sen_words:
                s_total[hin_word] = 0
                eng_sen_words = en_sentences[index_sen].split(" ")
                for eng_word in eng_sen_words:
                        s_total[hin_word] += translation_prob[hin_word][eng_word]
            
            #collect counts
            for hin_word in hin_sen_words:
                eng_sen_words = en_sentences[index_sen].split(" ")
                for eng_word in eng_sen_words:
                        count[(hin_word, eng_word)] += translation_prob[hin_word][eng_word]/s_total[hin_word]
                        total[eng_word] += translation_prob[hin_word][eng_word]/s_total[hin_word]                   

        #estimate probabilities
        for (hin_word, eng_word) in count.keys():
                translation_prob[hin_word][eng_word] = count[(hin_word, eng_word)]/total[eng_word]

        if is_converged(translation_prob, translation_prob_prev, epoch) == True:
            break
        
        translation_prob_prev = translation_prob
        
        
    return translation_prob


In [7]:
def train_model(sentences_train_en, sentences_train_hi):
    
    translation_prob = perform_EM(sentences_train_en, sentences_train_hi)
    return translation_prob


In [14]:
def test_model(dataset, tef):
#     tef = np.load('./models/IBMmodel1tef.npy')
    file = open("dev_version2.txt", 'w+')
    translated_data = []
    for sentence in dataset:
        translation = translate_sentence(sentence, tef, file)
        translated_data.append(translation)
    
    file.close()
    return translated_data

In [15]:
def translate_sentence(sentence, tef, file):
    '''
    takes the best translation of an hindi word
    and appends to eng sentence
    '''
    eng_sentence = []
    print("hin:",sentence)
    
    file.write("hin: ")
    file.write(sentence)
    file.write("\n")
    
    tokens = sentence.split(" ")
    for idx, token in enumerate(tokens):
#         print("token id, token", idx, token)
        max_score = -1
        max_eng_word = ""
        probabilities = list(tef[token].values())
        eng_words = list(tef[token].keys())
        for tef_index, prob in enumerate(probabilities):
            if prob > max_score:
                max_score = prob
                max_eng_word = eng_words[tef_index]
        
        eng_sentence.append(max_eng_word)
    
    eng_sentence = " ".join(eng_sentence)
    print("eng:", eng_sentence)
    file.write("eng: ")
    file.write(eng_sentence)
    file.write("\n\n")
    return eng_sentence


In [8]:
tef = train_model(sentences_train_en, sentences_train_hi)

epoch num: 1 

epoch num: 2 

epoch num: 3 

epoch num: 4 

epoch num: 5 

epoch num: 6 

epoch num: 7 

epoch num: 8 

epoch num: 9 

epoch num: 10 

epoch num: 11 

epoch num: 12 

epoch num: 13 

epoch num: 14 

epoch num: 15 



In [ ]:
# np.save("./models/IBMmodel1tef_3", tef)

In [29]:
for (fword, twtable) in tef.items():
    print('{0}'.format(fword))
    for (eword, prob) in sorted(twtable.items(), reverse=True, key=itemgetter(1)):
        if prob < 0.0001: continue # neglecting the extremely low possibilities
        print('{0}:{1:.4f}'.format(eword, prob))

मोतियाबिंद
cataract:0.3547
occular:0.1708
concessions:0.1074
compensation:0.0562
catarct:0.0419
disabilities:0.0406
patinets:0.0297
phaco:0.0165
intra:0.0056
mature:0.0014
का
bhonsle:0.7807
ka:0.7535
de:0.5083
citroen:0.4295
का:0.3743
pippali:0.3684
rassakasi:0.3646
chhin:0.3408
janmashtami:0.3359
agonizing:0.3341
teli:0.3256
megatemple:0.3210
chitarpur:0.3178
declension:0.3037
code:0.3000
suspecting:0.2974
temperament:0.2919
sar:0.2872
enjoy:0.2867
familes:0.2842
gosthal:0.2836
gaurimarg:0.2771
gada:0.2675
phimosis:0.2631
185:0.2598
tomb:0.2565
consume:0.2546
ranglo:0.2516
minimises:0.2515
pleasure:0.2462
target:0.2459
sweaty:0.2459
sahebganj:0.2453
sindoor:0.2419
barakopa:0.2407
collection:0.2406
argra:0.2398
bhadohi:0.2398
khurja:0.2398
chikan:0.2398
kunti:0.2393
's:0.2328
use:0.2326
enjoyed:0.2317
jhalmuri:0.2312
constructed:0.2282
cactus:0.2260
charm:0.2259
chauhans:0.2258
consecrate:0.2246
tanjung:0.2238
bungah:0.2238
bazat:0.2232
bottomless:0.2223
lakhi:0.2221
rupa:0.2217
falaud

rola:0.0233
sarhind:0.0232
doctors:0.0232
zebras:0.0232
ballabh:0.0230
lapland:0.0230
bahu:0.0229
main:0.0229
mahamastakabhishek:0.0229
level:0.0228
2637:0.0228
profits:0.0227
broadcasts:0.0227
rishiganga:0.0226
prahladhara:0.0226
naradkund:0.0226
panchateertha:0.0226
bhakhra:0.0226
son:0.0225
to-and-fro:0.0224
malaysian:0.0224
tirathgarh:0.0224
oesophagytis:0.0223
take:0.0223
mo:0.0222
1100:0.0222
firozabad:0.0222
riding:0.0221
pier:0.0221
heavy:0.0221
vidarbha:0.0221
christ:0.0221
jacksonville:0.0221
queue:0.0220
cancer:0.0220
amarkant:0.0220
hip:0.0219
thymus:0.0218
genitourinal:0.0218
andaman:0.0217
make:0.0217
shahjahanabad:0.0216
emerged:0.0215
akin:0.0215
metalicum:0.0215
recitation:0.0214
gopeshwar:0.0214
intoxication:0.0213
best:0.0213
invariably:0.0213
ligament:0.0213
bouldering:0.0212
freehand:0.0212
substances:0.0212
flamingoes:0.0212
aneket:0.0212
kavid:0.0212
inhabiting:0.0212
muni:0.0212
.:0.0211
tomatoes:0.0211
theory:0.0210
regiment:0.0209
raajpoot:0.0209
grandor:0.020

oedema:0.0018
afzal:0.0018
here:0.0018
gate:0.0018
young:0.0018
rice:0.0018
phenyl:0.0018
and:0.0018
creates:0.0018
option:0.0018
khandoli:0.0018
rectum:0.0018
setu:0.0018
expand:0.0018
smriti:0.0018
beauty:0.0017
ayurveda:0.0017
inwards:0.0017
hand:0.0017
access:0.0017
lovers:0.0017
dharmraj:0.0017
yudhishtir:0.0017
walking:0.0017
rajesh:0.0017
mother:0.0017
rs:0.0017
appropriate:0.0017
his:0.0017
following:0.0017
yercaud:0.0017
lord:0.0017
kesharia:0.0017
ghusel:0.0017
chaitravadi:0.0017
flock:0.0017
vapi:0.0017
attractive:0.0017
outlet:0.0017
i:0.0017
bright:0.0017
sagar:0.0017
enthralls:0.0017
indifferent:0.0017
chintamani:0.0017
rasraj:0.0017
ashwagandharishta:0.0017
saraswatarishta:0.0017
reservoir:0.0016
magic:0.0016
sight:0.0016
chandigarh:0.0016
museum:0.0016
low:0.0016
sherrington:0.0016
rob:0.0016
mixng:0.0016
825:0.0016
outer:0.0016
baji:0.0016
satellite:0.0016
serum:0.0016
eye:0.0016
prolapse:0.0016
enai:0.0016
udaipur:0.0016
mount:0.0016
lawns:0.0016
rapidly:0.0016
increa

23:0.0003
these:0.0003
punishments:0.0003
cornual:0.0003
simphaitum:0.0003
memoir:0.0003
n't:0.0003
spoon:0.0002
dandruff:0.0002
cruise:0.0002
कंप्यूटर:0.0002
से:0.0002
तरल:0.0002
पदार्थ:0.0002
प्रवाहित:0.0002
हैं:0.0002
रोगी:0.0002
तक:0.0002
दीर्घकालिक:0.0002
कूलिंग:0.0002
pads:0.0002
जिनमें:0.0002
जूड़ी:0.0002
पाइपों:0.0002
जरिए:0.0002
पैड:0.0002
साथ:0.0002
जुड़े:0.0002
होते:0.0002
जो:0.0002
उनमें:0.0002
कर:0.0002
तापमान:0.0002
एफ:0.0002
सी:0.0002
डिग्री:0.0002
रखता:0.0002
डॉक्:0.0002
टर:0.0002
दवाओं:0.0002
घंटे:0.0002
निद्रावस्था:0.0002
रखते:0.0002
raut:0.0002
spoilt:0.0002
chandraditya:0.0002
chandradita:0.0002
pratap:0.0002
acute:0.0002
maintained:0.0002
steel:0.0002
cell:0.0002
shikanji:0.0002
centre:0.0002
out:0.0002
test:0.0002
accumulation:0.0002
agreeable:0.0002
glonayan-30:0.0002
guns:0.0002
visits:0.0002
external:0.0002
ailment:0.0002
firing:0.0002
express:0.0002
chaunsath:0.0002
huge:0.0002
thick:0.0002
pre:0.0002
biophils:0.0002
grazing:0.0002
pause:0.0002
transmigration:0.

kinak:0.0001
mice:0.0001
lead:0.0001
उपचार
treatment:0.4204
treatments:0.3483
unnameable:0.2140
20,000:0.1894
treat:0.1861
treating:0.1821
trainee:0.1722
cure:0.1593
kaileuria:0.1525
restricy:0.1392
bhramari:0.1341
treats:0.1331
tretment:0.1211
preservative:0.1185
blemish:0.1133
ratnagiri:0.1109
materialistsc:0.1051
surjana:0.1030
marnom:0.1028
therapeutic:0.0977
treated:0.0944
cooperate:0.0885
aid:0.0883
dieticians:0.0852
varieities:0.0808
depths:0.0791
remedies:0.0787
colic:0.0760
retention:0.0687
enquiring:0.0675
misers:0.0640
shoul:0.0640
dbe:0.0640
scratches:0.0621
infinitesimal:0.0582
correction:0.0582
endurance:0.0527
nsaid:0.0455
mediums:0.0450
ailment:0.0448
cotrimoxazole:0.0428
unbridled:0.0326
artha:0.0324
therapy:0.0320
aak:0.0309
hostility:0.0304
newtek:0.0268
mediworld:0.0268
wlang:0.0264
eighty:0.0260
wrinkle:0.0254
creating:0.0235
immense:0.0234
proxima:0.0228
hyper:0.0221
embryo:0.0201
kartikeya:0.0178
pranayams:0.0171
anulom:0.0171
vilom:0.0171
bhrasika:0.0171
permane

...

polluted:0.0155
90:0.0140
110:0.0136
found:0.0120
butterflies:0.0108
fertile:0.0087
fish:0.0073
ants:0.0057
creatures:0.0054
fragrant:0.0046
abundance:0.0042
discovered:0.0037
potassium:0.0035
valuable:0.0025
caves:0.0021
orchids:0.0018
population:0.0013
complete:0.0008
shivlinga:0.0007
carotene:0.0007
carved:0.0005
digest:0.0002
species:0.0001
सभी
everyone:0.4421
all:0.4011
jundbedastar:0.3263
everybody:0.2465
hukka:0.2252
chilum:0.2252
gadhgaj:0.1788
determination:0.1746
knocked:0.1609
hajo:0.1458
taranjbeen:0.1397
khurasani:0.1397
upliftment:0.1361
chita:0.1336
marasmus:0.1279
1638:0.1269
napalese:0.1252
khukhri:0.1252
irritants:0.1244
strata:0.1192
lays:0.1171
surya:0.1133
90,000:0.1104
import:0.1032
menand:0.1021
mayur-asana:0.0984
destructive:0.0950
al:0.0937
everything:0.0899
fortunate:0.0870
groves:0.0869
warrior:0.0863
gurch:0.0841
gentian:0.0841
chirayata:0.0841
afatmoon:0.0841
boarding:0.0819
taxes:0.0815
varun:0.0809
pashanbhed:0.0809
gokshur:0.0809
yavakshar:0.0809
recepti

soaking:0.0599
overlook:0.0598
quietly:0.0596
lenses:0.0596
ling:0.0595
castles:0.0595
hid:0.0594
lifeline:0.0594
lau:0.0593
infront:0.0592
session:0.0592
permanence:0.0592
telegraph:0.0592
neurasthenia:0.0591
vomits:0.0590
protect:0.0588
mail:0.0588
weiss:0.0588
sceneries:0.0588
laundary:0.0587
alphabetical:0.0587
fulfil:0.0586
achievements:0.0585
calms:0.0585
mutatingt:0.0585
tambakhaani:0.0585
annacarolina:0.0584
jumping:0.0584
synthesizer:0.0583
affordable:0.0582
naugaon:0.0582
430:0.0582
urdhvabahu:0.0579
anybody:0.0578
reflect:0.0578
scarf:0.0578
lemongrass:0.0578
decorates:0.0577
subjugate:0.0576
reticular:0.0576
transplanted:0.0575
louvre:0.0574
roadside:0.0574
koop:0.0574
crows:0.0573
mahashivaratri:0.0573
openly:0.0572
correctly:0.0572
hammering:0.0569
pitton:0.0569
ardhachandrasana:0.0567
cassette:0.0566
sit:0.0562
relaxing:0.0561
horsemen:0.0560
boundry:0.0559
widened:0.0559
raods:0.0559
four-lane:0.0559
opioid:0.0559
receptors:0.0559
hurly:0.0559
burly:0.0559
chatrungi:0.0

sandals:0.0030
whale:0.0030
bag:0.0029
www:0.0029
advancement:0.0029
merry:0.0029
consult:0.0029
emerged:0.0029
acne:0.0028
unfolding:0.0028
made:0.0028
process:0.0028
grass:0.0028
parshuram:0.0028
titters:0.0028
baltal:0.0027
yogic:0.0027
cruise:0.0027
holi:0.0027
waist:0.0027
immediately:0.0027
comfort:0.0027
illegal:0.0027
slave:0.0026
ascend:0.0026
dangers:0.0026
partner:0.0026
parassinikkadavu:0.0026
sree:0.0026
clay:0.0026
reactions:0.0026
fomentation:0.0026
lamps:0.0026
vati:0.0026
sightseeing:0.0025
legislative:0.0025
half:0.0025
lighten:0.0025
doshas:0.0025
getting:0.0024
countless:0.0024
meriamun:0.0024
mizoram:0.0024
vessels:0.0024
controlling:0.0023
swahili:0.0023
reducing:0.0023
adjust:0.0023
asking:0.0023
quaint:0.0023
exploitation:0.0023
situation:0.0023
executed:0.0023
training:0.0023
weather:0.0023
affection:0.0022
ancestry:0.0022
assamese:0.0022
computer:0.0022
avenue:0.0022
pawan:0.0022
behalf:0.0022
upon:0.0022
cover:0.0022
back:0.0022
ventricle:0.0022
nutritive:0.0

automatically:0.0002
nutritionist:0.0002
testing:0.0002
compel:0.0002
voice:0.0002
remove:0.0002
matter:0.0002
beside:0.0002
urinate:0.0002
down:0.0002
camped:0.0002
motor:0.0002
gastric:0.0002
trail:0.0002
international:0.0002
upto:0.0002
ice:0.0002
cancer:0.0002
ovulation:0.0002
unnecessary:0.0002
kansuwa:0.0002
chapatti:0.0002
jat:0.0002
2700:0.0002
sensation:0.0002
hardeep:0.0002
stretching:0.0002
infested:0.0002
chew:0.0002
reforms:0.0002
balancing:0.0002
normal:0.0002
untiring:0.0002
answering:0.0002
devraj:0.0002
dhoti:0.0002
unique:0.0002
jalpaiguri:0.0002
vehicles:0.0002
ineffective:0.0002
worshipped:0.0002
greeting:0.0002
digests:0.0002
prevents:0.0002
surviving:0.0002
permanently:0.0002
thalassery:0.0002
filling:0.0002
dependent:0.0002
toxins:0.0002
holiday:0.0002
masters:0.0002
cemented:0.0001
monsoons:0.0001
repair:0.0001
sips:0.0001
anemia:0.0001
with:0.0001
thrown:0.0001
heads:0.0001
industries:0.0001
topical:0.0001
balance:0.0001
minute:0.0001
others:0.0001
losses:0.000

KeyboardInterrupt: 

In [17]:
# print(tef['तथा']['and'])

In [18]:
predicted_translations = test_model(sentences_dev_hi, tef)

hin: बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।
eng: rather you  from also chained vacancies .
hin: लिवरपूल यूनिवर्सिटी के आप्थैमोलॉजी विभाग के अध्यक्ष व विख्यात नेत्ररोग शोधकर्ता डॉ. इयान ग्रीयरसन कहते हैं ।
eng:  university के  department के fazal riches wool  researcher dr   call are .
hin: कि खाने में विटामिन-सी और ओमेगा-3 तेलों व कुछ विशेष पौधों की मात्रा बढ़ाने से आमतौर पर अंधेपन का कारण बनने वाले मोतियाबिंद व ग्लूकोमा जैसे रोगों से बचा जा सकता है ।
eng: that eating in dosctors and maker oils riches few special plants ki quantity appeal from normally on blur bhonsle cause forming reptiles cataract riches glaucoma like thothahar from frss south-eastern can is .
hin: तो आइए जानते है कि नजर को स्वस्थ बनाए रखने के लिए कैसा हो आपका आहार ।
eng: then lets knew is that visible shukla healthy cairns harbouring के for eleme occur crave diet .
hin: नजर तेज बनी रहे , इसके लिए आपको चाहिए की अपने आहार में आप हरी सब्जियाँ शामिल करें ।
eng: visible lound ashtadhatu constables , creativity for himsle

eng: onikomysis what are ?
hin: जब स्थिति और सुधर जाए तो अभ्यास में जानुशिरासन , सुप्तवज्रासन , त्रिकोणासन , उत्तानपादासन , गोमुखासन तथा अर्धमत्स्येंद्रासन आदि को जोड़ लें ।
eng: when situation and withor corpulence then practice in janushirasana ,  , trikonasana , uttanapadasana , gomukhasana kailas  etc shukla symphytum grind .
hin: यहाँ पर जानुशिरासन के अभ्यास की विधि प्रस्तुत है - दोनों पैरों को सामने की ओर फैलाकर बैठ जाएँ ।
eng: here on janushirasana के practice ki method ghumar is - both legs shukla front ki side laterally strewn mirisa .
hin: दाएँ पैर को घुटने से मोड़कर इसके तलवे का बाएँ पैर की जंघा से सटा दें तथा एड़ी जननेंद्रिय के नीचे रखें ।
eng: auricle leg shukla knee from folding creativity soles bhonsle sarpasana leg ki thigh from parkota let kailas heel anywhere के below keep .
hin: दोनों हाथों को बाएँ पैर के पंजे के पास ले जाते हुए आगे की ओर इतना झुकें कि सिर जमीन को स्पर्श करे ।
eng: both hands shukla sarpasana leg के claws के aalam jacuzzi quicker curvy ahead ki side 

eng: gnerally tension from hormone  membranes के prefrontal shukla serious harm dianmine is .
hin: अच्छी नींद लेने से दिमाग तरोताजा रहता है ।
eng: tarah sleep relishing from membranes refreshes stays is .
hin: नींद पूरी नहीं हो पाने से दिनभर थकावट रहती है और किसी काम में ध्यान केन्द्रित करना मुश्किल हो जाता है ।
eng: sleep fulfilled not occur circulating from alighting fatigue kunain is and someone work in attention centric diffucult piragarhi occur known is .
hin: सिगरेट पीने से मस्तिष्क तक आक्सीजन पहुँचाने वाली धमनियाँ सिकुड़ने लगती है जिससे दिमाग कमजोर होने लगता है ।
eng: cigarette drinking from brain till obtains convey yumthangchu estrogens shrivel growling is pans membranes weak occurrence held is .
hin: फल , सब्जियों का पर्याप्त मात्रा में सेवन सेहत के साथ दिमाग के लिए भी काफी लाभकारी साबित होता है ।
eng: fruit , vegetables bhonsle manufactures quantity in consumption disbeliefs के alongside membranes के for also quite bneficial prove fibre is .
hin: विटामिन बी , बी-12 , बी-6 , फ

eng: apeej four younger hole ekvira occurrence के cause inn wounds के inflaming or infected occur going ki scope quite less occur critical is .
hin: सामान्य शल्यक्रिया के पश्‍चात मरीज करीब एक सप्ताह बिस्तर पर ही रहता है ।
eng: normal amputation के posibility detached nearly one weeks bed on barhait stays is .
hin: ऐसे में उसे निमोनिया होने , शिराओं में खून के थक्के जमने जैसी लताओं का सामना करना पड़ सकता है ।
eng: such in him pneumonia occurrence , capillary in bleeding के clots deposition inhibitors bohra bhonsle encountered diffucult anaestesia can is .
hin: लैपरोस्कोपिक विधि में ये संभावनाएँ नगण्य होती हैं ।
eng:  method in skiiing possibilities null sundown are .
hin: लैपरोस्कोपिक विधि में सामान्य चिकित्सा के मुकाबले दोगुना समय लगता है ।
eng:  method in normal therapy के versus mishti time held is .
hin: मरीज को ज्यादा निश्चेतक दवा देनी पड़ती है , जिसके अपने नुकसान हैं ।
eng: detached shukla gnerally  dava rasa ahemdabad is , havankunda cd harm are .
hin: शल्यचिकित्सा के क्षेत्र में ’

eng: 1,20,00,000 programs in everyone governmnet ,  , voluntary organizations bhonsle active co-operation explored south-eastern remained is .
hin: दृष्टिहीनता की व्यापकता दर को कम करने के लिये भारत सरकार द्वारा नेत्र विभाग से सम्बन्धित स्टॉफ को विभिन्न स्तर पर प्रशिक्षण दिया जाता है ।
eng:  ki pervasiveness rate shukla less doing के tilwaraghat india government samvat 1,20,00,000 department from self-breast  shukla various level on training gave known is .
hin: दृष्टिहीनता के मुख्य कारणों में मोतियाबिंद 55 प्रतिशत है , दृष्टि दोष 19 प्रतिशत , काला मोतिया 4 प्रतिशत , कॉर्नियल पैथॉलाजी 7 प्रतिशत , अन्य 15 प्रतिशत है ।
eng:  के main reasons in cataract 55 percent is , evaluated dosha 19 percent , amb cataract 4 percent , corneal  7 percent , other 15 percent is .
hin: जब साधारणतया स्वच्छ लैन्स या उसका कैपस्यूल धुंधला या दुधिया हो जाता है जिससे रोशनी आँख के अन्दर नही पहुँच पाती है तथा मरीज को धुंधला या कम दिखाई देता है ।
eng: when dizzines clean capsular or mounts capsular blurring or  oc

eng: rajasthan के royal bar-be-que in holidays spending bathers in from panchmandir people researcher  के  ki stroll ki stickiness ! away why moments , jaipur bathers in from also character researcher barhait perhaps vadpradaka mattupetty in night spent stickiness .
hin: अमन बाग के खुलने के बाद वहाँ देश - विदेश के बड़े-बड़े लोग पहुँचे हैं ।
eng:  bagh के opening के after therefrom country - abroad के alpatthar people banka are .
hin: मुझे वो रात भी याद है जब शबाना आजमी ने मुझे तहेदिल से वहाँ बुलाया था पर मैं नहीं जा सकी थी ।
eng: me stilt night also remember is when   researcher me  from therefrom speaker was on i not south-eastern explorations 242 .
hin: यह बात अलग है कि वे खुद राह भटक कर भानगढ़ पहुँच गयी थीं ।
eng: this thing distinct is that they yourself painimath wanders taxes  accessibility khalatika mrigthuni .
hin: इस बार अचानक अपनी विदेशी मेहमान की दावत पर मैं वहाँ पहुँची , तो लगा कि जो लोग कम समय में एक शानदार छुट्‍टी बिताना चाहते हैं , उन्हें तो अमन बाग जरूर जाना चाहिए ।
eng: t

eng: incidentally , advance in tickets jewellery cheapest varnigad is , therefore last  in bus atique modified doing from pletenberg is that nitin bit earlier pilates .
hin: टू वे टिकट की बजाय राउंड ट्रिप के बारे में पता करें ।
eng: bouncing they tickets ki instead bob trip के about in detected pilates .
hin: ये सस्ती पड़ती हैं ।
eng: skiiing cheap ahemdabad are .
hin: बैगेज को लेकर नियम खासे कड़े होते हैं , इसलिए सामान के साइज व वजन के बारे में पूछताछ कर लें ।
eng:  shukla 1846 rule southern-western shampoos kalal are , therefore stuff के criticised riches weight के about in 01902 taxes grind .
hin: कहीं ऐसा न हो कि आपको अपना कुछ सामान टर्मिनल पर छोड़ना पड़े या फिर एक्स्ट्रा फीस भरनी पड़े ।
eng: somewhere bhagirathji neither occur that himslef defend few stuff terminal on partition dyscoria or descension pedometer fees  dyscoria .
hin: साथ ही टूर कंपनी से इंश्योरेंस या नुकसान की भरपाई के नियम भी जान लें ।
eng: alongside barhait atique company from  or harm ki compensation के rule also live

eng: nickname is that this sarak rome bhonsle main marg is .
hin: यह अद्वितीय इमारत ’ कैपिटोलाइन हिल ’ पर बनी है और काफी ऊँचाई पर स्थित है ।
eng: this dungse building `  morahabadi ` on ashtadhatu is and quite height on located is .
hin: इस पर चढ़ने की सीढ़ियों को मशहूर मूर्तिकार माइकल एंजलो ने 1536 में बनाया था ।
eng: this on unreliable ki leaping shukla nurseries sriniketan   researcher kanada in krishn was .
hin: कैपिटोलाइन के सामने घोड़े पर सवार सम्राट मार्कस ओरेलिअस की पीतल की मूर्ति है , जिन्होंने 160 - 180 ई. तक रोम पर शासन किया था ।
eng:  के front horse on riding emperor   ki brass ki idol is , greeks 160 - 180 399 till rome on reign veda was .
hin: यह भव्य मूर्ति आपको आश्‍चर्यचकित कर देगी ।
eng: this magnificent idol himslef toursist taxes guage .
hin: इसके अलावा कैपिटोलाइन पर बना टॉवर भी इस इमारत की अलग पहचान है ।
eng: creativity apart  on dehradun-chakrata tower also this building ki distinct identity is .
hin: ’ कैपिटोलाइन कॉम्प्लेक्स ’ में एक चर्च , म्यूजियम और पुराना मंदिर भ

eng: popular himalayas malls के behind located and promised price on ultramodern facilities riches multicoloured ambience from containing conditioned rooms wala kaejput hotel undestand this journey shukla memorable dehradun-chakrata ties is .
hin: आप अहमदाबाद में साबरमती आश्रम , साइंस सिटी , अक्षरधाम मंदिर और प्रसिद्ध वैष्णों देवी मंदिर की प्रतिकृति जैसे दर्शनीय स्थलों का भी भ्रमण कर सकते हैं ।
eng: you ahmedabad in sabarmati ashram , science northeastern , baha'i mandir and famous vaishno devi mandir ki chopak like worth-seeing sites bhonsle also excursion taxes instantaneous are .
hin: एक्सप्रेस हाइवे की अनूठी ड्राइव के बाद आप वडोदरा पहुँच सकते हैं जिसे बड़ौदा भी कहा जाता है जो समृद्ध परंपरा से युक्‍त एक कास्मोपोलिटन शहर है ।
eng: express highways ki kiranji drive के after you vadodara accessibility instantaneous are leni  also said known is which enrich tradition from containing one  city is .
hin: यहाँ फतेहगंज कैंप रोड पर स्थित जिंजर होटल के रेस्टोरेंट बफे जंक्शन में स्वादिष्‍ट ब्रे

In [11]:
def find_recall_precision(predicted_translations):
    y_pred = list(predicted_translations)
    y_true = sentences_dev_hi
    recall = recall(y_true, y_pred, average=None)
    precision = precision(y_true, y_pred, average=None)
    return recall, precision;